# Construção dos Datasets 

## Projeto de Mapeamento Marinho Apresentado Nas Jornadas de Engenharia Hidrográfica

## Diogo Ceddia Porto Silva

#### Importação de bibliotecas

In [1]:
import pandas as pd
import numpy as np
from pyproj import Proj
import warnings
warnings.filterwarnings("ignore")
from pykdtree.kdtree import KDTree
import time

#### Carregando dados Brutos

In [2]:
# Carregando todos os arquivos BATIMÉTRICOS de todas as resoluções.
df_bat_2m = pd.read_csv("C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\Madeira_13e14_WGS84_UTM28N_2m.txt",
                      skiprows=1,delim_whitespace=True,names=['utmx','utmy','z'])
df_bat_2m['z_res'] = '2m'

df_bat_4m = pd.read_csv("C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\Madeira_13e14_WGS84_UTM28N_4m.txt",
                      skiprows=1,delim_whitespace=True,names=['utmx','utmy','z'])
df_bat_4m['z_res'] = '4m'

# Criando arquivo único da batimetria, com coluna associada à resolução.
df_bat = pd.concat([df_bat_2m,df_bat_4m],axis=0)
del df_bat_2m,df_bat_4m

# Carregando dado SÍSMICO
df_sbp = pd.read_csv(r"C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\Thickness_MadeiraNW.txt",sep=';')

# Convertendo de projeção geográfica WGS84 para UTM WGS84
df_sbp['LATITUDE'] = df_sbp['LATITUDE'].str.replace(',','.')
df_sbp['LATITUDE'] = df_sbp['LATITUDE'].astype("float")
df_sbp['LONGITUDE'] = df_sbp['LONGITUDE'].str.replace(',','.')
df_sbp['LONGITUDE'] = df_sbp['LONGITUDE'].astype("float")
df_sbp['THICKNESS_'] = df_sbp['THICKNESS_'].str.replace(',','.')
df_sbp['THICKNESS_'] = df_sbp['THICKNESS_'].astype("float")
df_sbp['classe'] = 1
df_sbp['classe'][df_sbp[df_sbp['THICKNESS_']!=0].index] = 0

myProj = Proj("+proj=utm +zone=28 +north +ellps=WGS84 +datum=WGS84 +units=m")
utmx, utmy = myProj(df_sbp.LONGITUDE, df_sbp.LATITUDE) 
df_sbp['utmx'] = utmx
df_sbp['utmy'] = utmy

# Pegando somente colunas de interesse da sísmica
df_sbp = df_sbp[['utmx','utmy','classe']]
del utmx,utmy

# Carregando dados de BACKSCATTER
df_bs = pd.read_csv(r"C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\backscatter.txt",names = ['utmx','utmy','bs'])

# Carregando dados de DECLIVIDADE DO FUNDO
df_sl = pd.read_csv(r"C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\slope.xyz",names = ['utmx','utmy','sl'])

# Carregando dados de ORIENTAÇÃO DA DECLIVIDADE DO FUNDO
df_as = pd.read_csv(r"C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\aspect.xyz",names = ['utmx','utmy','as'])

# Carregando dados da DECLIVIDADE SUBMARINA
df_ds = pd.read_csv(r"C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\declive submarino.txt",usecols=[0,1],names = ['utmx','utmy'])

# Carregando dados de LINHA DE COSTA
df_lc = pd.read_csv(r"C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\linha de costa.txt",usecols=[0,1],names = ['utmx','utmy'])

In [3]:
# Pegando distância do declive submarino e a linha de costa. 
kd_tree = KDTree(df_lc[['utmx','utmy']].values)
start = time.time()
dist, idx = kd_tree.query(df_ds[['utmx','utmy']].values, k=1)
end = time.time()
print(end - start)
df_ds['dist_lc'] = dist

0.003000497817993164


#### Construido Dataset de Treino/Validação

In [4]:
# PYKDTREE

data = df_sbp

# Correlacionando sísmica com batimetria
kd_tree = KDTree(df_bat[['utmx','utmy']].values)
start = time.time()
dist, idx = kd_tree.query(df_sbp[['utmx','utmy']].values, k=1)
end = time.time()
print(end - start)
# Salvando correlação de batimetria no dataset final
data['z'] = df_bat.z.iloc[idx].values
data['z_res'] = df_bat.z_res.iloc[idx].values
data['z_dist'] = dist

# Correlacionando sísmica com backscatter
kd_tree = KDTree(df_bs[['utmx','utmy']].values)
start = time.time()
dist, idx = kd_tree.query(df_sbp[['utmx','utmy']].values, k=1)
end = time.time()
print(end - start)
# Salvando correlação de batimetria no dataset final
data['bs'] = df_bs.bs.iloc[idx].values
data['bs_dist'] = dist

# Correlacionando sísmica com declividade do fundo
kd_tree = KDTree(df_sl[['utmx','utmy']].values)
start = time.time()
dist, idx = kd_tree.query(df_sbp[['utmx','utmy']].values, k=1)
end = time.time()
print(end - start)
# Salvando correlação de declividade do fundo no dataset final
data['sl'] = df_sl.sl.iloc[idx].values
data['sl_dist'] = dist

# Correlacionando sísmica com orientação da declividade do fundo
kd_tree = KDTree(df_as[['utmx','utmy']].values)
start = time.time()
dist, idx = kd_tree.query(df_sbp[['utmx','utmy']].values, k=1)
end = time.time()
print(end - start)
# Salvando correlação de declividade do fundo no dataset final
data['as'] = df_as['as'].iloc[idx].values
data['as_dist'] = dist

# DISTANCIA EUCLIDIANA DA LINHA DE COSTA
kd_tree = KDTree(df_lc[['utmx','utmy']].values)
start = time.time()
dist, idx = kd_tree.query(df_sbp[['utmx','utmy']].values, k=1)
end = time.time()
print(end - start)
data['dist_lc'] = dist

# DISTANCIA EUCLIDIANA DA DECLIVE SUBMARINO
kd_tree = KDTree(df_ds[['utmx','utmy']].values)
start = time.time()
dist, idx = kd_tree.query(df_sbp[['utmx','utmy']].values, k=1)
end = time.time()
print(end - start)
data = data.reset_index(drop=True)
data['dist_ds'] = dist
#data['dist_ds'] = data['dist_ds']*(np.sign(data['dist_ds'] - df_ds['dist_lc'].iloc[idx].reset_index(drop=True)))
#data['dist_ds_lc'] = df_ds['dist_lc'].iloc[idx].reset_index(drop=True)

0.1356348991394043
0.13068413734436035
0.16422748565673828
0.18649911880493164
0.10423040390014648
0.11671328544616699


#### Construindo Dataset de Extrapolação

In [5]:
# PYKDTREE

data_all = df_bat

# Correlacionando BATIMETRIA COM BACKSCATTER
kd_tree = KDTree(df_bs[['utmx','utmy']].values)
start = time.time()
dist, idx = kd_tree.query(df_bat[['utmx','utmy']].values, k=1)
end = time.time()
print(end - start)
data_all['bs'] = df_bs.bs.iloc[idx].values
data_all['bs_dist'] = dist

# Correlacionando BATIMETRIA COM DECLIVIDADE DO FUNDO
kd_tree = KDTree(df_sl[['utmx','utmy']].values)
start = time.time()
dist, idx = kd_tree.query(df_bat[['utmx','utmy']].values, k=1)
end = time.time()
print(end - start)
data_all['sl'] = df_sl.sl.iloc[idx].values
data_all['sl_dist'] = dist

# Correlacionando BATIMETRIA COM ORIENTAÇÃO DA DECLIVIDADE DO FUNDO
kd_tree = KDTree(df_as[['utmx','utmy']].values)
start = time.time()
dist, idx = kd_tree.query(df_bat[['utmx','utmy']].values, k=1)
end = time.time()
print(end - start)
data_all['as'] = df_as['as'].iloc[idx].values
data_all['as_dist'] = dist

# DISTANCIA EUCLIDIANA DA LINHA DE COSTA
kd_tree = KDTree(df_lc[['utmx','utmy']].values)
start = time.time()
dist, idx = kd_tree.query(df_bat[['utmx','utmy']].values, k=1)
end = time.time()
print(end - start)
data_all['dist_lc'] = dist

# DISTANCIA EUCLIDIANA DA DECLIVE SUBMARINO
kd_tree = KDTree(df_ds[['utmx','utmy']].values)
start = time.time()
dist, idx = kd_tree.query(df_bat[['utmx','utmy']].values, k=1)
end = time.time()
print(end - start)
data_all['dist_ds'] = dist
data_all = data_all.reset_index(drop=True)
#data_all['dist_ds'] = data_all['dist_ds']*(np.sign(data_all['dist_ds'] - df_ds['dist_lc'].iloc[idx].reset_index(drop=True)))
#data_all['dist_ds_lc'] = df_ds['dist_lc'].iloc[idx].reset_index(drop=True)

del df_bat, df_bs, df_ds, df_lc, df_sbp, df_sl, dist, start, end, idx, kd_tree, df_as

2.2078871726989746
2.489129066467285
2.500565767288208
2.0352015495300293
2.1158087253570557


#### Filtrando todo atributo que dista 10m+ do grid pivot.

In [6]:
data = data.reset_index(drop=True)
data = data.drop(np.where(data.z_dist>10)[0]).reset_index(drop=True)
data = data.drop(np.where(data.bs_dist>10)[0]).reset_index(drop=True)
data = data.drop(np.where(data.sl_dist>10)[0]).reset_index(drop=True)
data = data.drop(np.where(data.as_dist>10)[0]).reset_index(drop=True)
data_all = data_all.reset_index(drop=True)
data_all = data_all.drop(np.where(data_all.bs_dist>10)[0]).reset_index(drop=True)
data_all = data_all.drop(np.where(data_all.sl_dist>10)[0]).reset_index(drop=True)
data_all = data_all.drop(np.where(data_all.as_dist>10)[0]).reset_index(drop=True)

#### Filtrando dados duplicados

In [7]:
#print('\nQuantidade de linhas no dataset original:',len(data))
#duplicateDFRow = data[data.duplicated(subset=['z','bs','sl','as','classe'])]
#print('Quantidade de linhas duplicadas:', len(duplicateDFRow))
#print('% filtrado:', np.round(len(duplicateDFRow)/len(data)*100,3))
#data.drop(axis=0, index=duplicateDFRow.index, inplace=True)
#data = data.reset_index(drop=True)
#print('Quantidade de linhas do novo dataset:',len(data),'\n')
#del duplicateDFRow
#print('% de sedimento:',np.round((len(np.where(data.classe==0)[0])/len(data))*100,3))
#print('% de rocha:',np.round((len(np.where(data.classe==1)[0])/len(data))*100,3))

In [8]:
print('\nQuantidade de linhas no dataset original:',len(data_all))
duplicateDFRow = data_all[data_all.duplicated(subset=['z','bs','sl','as'])]
print('Quantidade de linhas duplicadas:', len(duplicateDFRow))
print('% filtrado:', np.round(len(duplicateDFRow)/len(data_all)*100,3))
data_all.drop(axis=0, index=duplicateDFRow.index, inplace=True)
data_all = data_all.reset_index(drop=True)
print('Quantidade de linhas do novo dataset:',len(data_all),'\n')
del duplicateDFRow


Quantidade de linhas no dataset original: 5925070
Quantidade de linhas duplicadas: 5950
% filtrado: 0.1
Quantidade de linhas do novo dataset: 5919120 



#### Filtrando dados que estão no declive submarino

In [9]:
#data = data.drop(np.where(data['dist_ds']<50)[0]).reset_index(drop=True)
#data_all = data_all.drop(np.where(data_all['dist_ds']<50)[0]).reset_index(drop=True)

#### Descrição e detalhamento dos dados

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191598 entries, 0 to 191597
Data columns (total 14 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   utmx     191598 non-null  float64
 1   utmy     191598 non-null  float64
 2   classe   191598 non-null  int64  
 3   z        191598 non-null  float64
 4   z_res    191598 non-null  object 
 5   z_dist   191598 non-null  float64
 6   bs       191598 non-null  float64
 7   bs_dist  191598 non-null  float64
 8   sl       191598 non-null  float64
 9   sl_dist  191598 non-null  float64
 10  as       191598 non-null  float64
 11  as_dist  191598 non-null  float64
 12  dist_lc  191598 non-null  float64
 13  dist_ds  191598 non-null  float64
dtypes: float64(12), int64(1), object(1)
memory usage: 20.5+ MB


In [11]:
data.describe()

,utmx,utmy,classe,z,z_dist,bs,bs_dist,sl,sl_dist,as,as_dist,dist_lc,dist_ds
count,191598.000000,1.915980e+05,191598.000000,191598.000000,191598.000000,191598.000000,191598.000000,191598.000000,191598.000000,191598.000000,191598.000000,191598.000000,191598.000000
mean,285717.867186,3.629459e+06,0.400114,70.553797,1.362341,-19.806302,2.167814,2.427746,0.770287,112.430045,0.770287,3523.354167,1998.196859
std,2644.643443,2.303986e+03,0.489922,14.035168,0.662312,6.531755,0.809359,5.550080,0.323013,83.926307,0.323013,2140.623294,1336.726379
min,280127.962030,3.623894e+06,0.000000,39.346000,0.005770,-41.341000,0.005050,0.000000,0.005770,0.000000,0.005770,715.801462,2.491801
25%,282641.572615,3.627810e+06,0.000000,60.659000,0.867293,-24.971000,1.595921,0.650000,0.565119,57.000000,0.565119,1850.328125,911.224655
50%,286673.440596,3.629665e+06,0.000000,70.911000,1.335847,-20.870000,2.258685,1.140000,0.800351,82.000000,0.800351,2632.794801,1618.190985
75%,287695.587580,3.631278e+06,1.000000,78.198000,1.855233,-13.310000,2.769188,1.860000,0.978651,145.000000,0.978651,5472.919231,3140.065377
max,290013.835834,3.633793e+06,1.000000,149.974000,9.988894,2.119000,6.576014,89.000000,9.856238,359.000000,9.856238,8561.918000,4824.692303


In [12]:
data_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5919120 entries, 0 to 5919119
Data columns (total 12 columns):
 #   Column   Dtype  
---  ------   -----  
 0   utmx     float64
 1   utmy     float64
 2   z        float64
 3   z_res    object 
 4   bs       float64
 5   bs_dist  float64
 6   sl       float64
 7   sl_dist  float64
 8   as       float64
 9   as_dist  float64
 10  dist_lc  float64
 11  dist_ds  float64
dtypes: float64(11), object(1)
memory usage: 541.9+ MB


In [13]:
data_all.describe()

,utmx,utmy,z,bs,bs_dist,sl,sl_dist,as,as_dist,dist_lc,dist_ds
count,5.919120e+06,5.919120e+06,5.919120e+06,5.919120e+06,5.919120e+06,5.919120e+06,5919120.0,5.919120e+06,5919120.0,5.919120e+06,5.919120e+06
mean,2.859595e+05,3.630915e+06,7.058757e+01,-1.916879e+01,2.163977e+00,2.974167e+00,0.0,1.275024e+02,0.0,3.289966e+03,1.733538e+03
std,2.457484e+03,3.240108e+03,1.715595e+01,6.409923e+00,8.056143e-01,5.509527e+00,0.0,8.354982e+01,0.0,1.938720e+03,1.209329e+03
min,2.801360e+05,3.623904e+06,3.003900e+01,-4.859100e+01,5.656854e-03,0.000000e+00,0.0,0.000000e+00,0.0,7.133646e+02,1.221843e-01
25%,2.841280e+05,3.628392e+06,5.670000e+01,-2.434100e+01,1.595724e+00,7.700000e-01,0.0,6.500000e+01,0.0,1.729975e+03,8.452185e+02
50%,2.864400e+05,3.630768e+06,7.089100e+01,-1.961000e+01,2.256461e+00,1.370000e+00,0.0,1.110000e+02,0.0,2.662756e+03,1.458822e+03
75%,2.878880e+05,3.633524e+06,7.965200e+01,-1.331000e+01,2.763794e+00,2.410000e+00,0.0,1.650000e+02,0.0,4.482468e+03,2.518825e+03
max,2.902520e+05,3.637124e+06,1.499990e+02,7.479000e+00,8.148684e+00,8.935000e+01,0.0,3.590000e+02,0.0,8.662566e+03,4.853595e+03


In [14]:
data_all.head()

,utmx,utmy,z,z_res,bs,bs_dist,sl,sl_dist,as,as_dist,dist_lc,dist_ds
0,289984.0,3624500.0,59.993,2m,-23.081,2.618635,1.18,0.0,31.0,0.0,2152.248438,1709.201926
1,289986.0,3624500.0,59.987,2m,-23.081,3.678213,88.09,0.0,4.0,0.0,2150.919495,1711.109044
2,289984.0,3624502.0,59.961,2m,-23.081,0.853960,1.27,0.0,56.0,0.0,2150.754683,1708.600241
3,289986.0,3624502.0,59.930,2m,-23.081,2.720523,88.09,0.0,20.0,0.0,2149.424817,1710.508029
4,289982.0,3624504.0,59.984,2m,-23.081,1.982233,1.36,0.0,62.0,0.0,2150.593579,1706.092438


#### Salvando

In [15]:
data.to_csv(r"C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\dado_linhas.txt",index=False,header=True)
data_all.to_csv(r"C:\DCPS\GitHub\Dados_MapeamentoIA_JEH\dado_tudo.txt",index=False,header=True)